# Installation of Packages

In [1]:
!! pip -q install keras_hub

[]

# Importing packages

In [20]:
import keras_hub
import tensorflow_datasets as tfds
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Download or Import Dataset

# Reading Data

In [3]:
data, dataset_info = tfds.load("cats_vs_dogs", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9


Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.1KNU4S_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


# Data Inspection

# Data Cleaning

# Visualization of data

# Data Preprocessing

In [4]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

In [5]:
train_dataset = data["train"]

In [6]:
num_classes = dataset_info.features["label"].num_classes

In [7]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
tfds.disable_progress_bar()

data, dataset_info = tfds.load("cats_vs_dogs", with_info=True, as_supervised=True)
train_steps_per_epoch = dataset_info.splits["train"].num_examples // BATCH_SIZE
train_dataset = data["train"]

num_classes = dataset_info.features["label"].num_classes

resizing = keras.layers.Resizing(
    IMAGE_SIZE[0], IMAGE_SIZE[1], crop_to_aspect_ratio=True
)


def preprocess_inputs(image, label):
    image = tf.cast(image, tf.float32)
    # Staticly resize images as we only iterate the dataset once.
    return resizing(image), tf.one_hot(label, num_classes)


# Shuffle the dataset to increase diversity of batches.
# 10*BATCH_SIZE follows the assumption that bigger machines can handle bigger
# shuffle buffers.
train_dataset = train_dataset.shuffle(
    10 * BATCH_SIZE, reshuffle_each_iteration=True
).map(preprocess_inputs, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)

# Custom Functions

# Define Model

In [8]:
model = keras_hub.models.ImageClassifier.from_preset(
    preset="resnet_v2_50_imagenet", 
    num_classes=2
)

# Compile and Fit

In [9]:
model.compile(
    optimizer = 'sgd', 
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [10]:
model.fit(train_dataset)

I0000 00:00:1730926739.014421     109 service.cc:145] XLA service 0x7de81c03c640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730926739.014520     109 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730926739.014530     109 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1730926759.845966     109 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


727/727 ━━━━━━━━━━━━━━━━━━━━ 267s 323ms/step - accuracy: 0.6565 - loss: 2.7885


# Visualization of model

# Evaluation

In [ ]:
filepath = keras.utils.get_file(
    origin="https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/5hR96puA_VA.jpg/1024px-5hR96puA_VA.jpg"
)
image = keras.utils.load_img(filepath)
plt.imshow(image)

In [ ]:
predictions = model.predict(image)
predictions

In [16]:
classes = {0: "cat", 1: "dog"}
print("Top class is:", classes[predictions[0].argmax()])

Top class is: dog


# Deploying Model

In [32]:
import kagglehub

In [37]:
preset_dir = "./image_classifier"
model.save_to_preset(preset_dir)

In [38]:
kaggle_uri = f"kaggle://asieduas/images/keras/image_classifier"
keras_hub.upload_preset(kaggle_uri, preset_dir)

Uploading Model https://www.kaggle.com/models/asieduas/images/keras/image_classifier ...
Model 'images' does not exist or access is forbidden for user 'asieduas'. Creating or handling Model...
Model 'images' Created.
Starting upload for file ./image_classifier/config.json


Uploading: 100%|██████████| 840/840 [00:00<00:00, 1.77kB/s]

Upload successful: ./image_classifier/config.json (840B)
Starting upload for file ./image_classifier/task.weights.h5


Uploading: 100%|██████████| 94.6M/94.6M [00:00<00:00, 101MB/s]

Upload successful: ./image_classifier/task.weights.h5 (90MB)
Starting upload for file ./image_classifier/image_converter.json


...

Uploading: 100%|██████████| 94.6M/94.6M [00:00<00:00, 108MB/s] 

Upload successful: ./image_classifier/model.weights.h5 (90MB)
Starting upload for file ./image_classifier/metadata.json


Uploading: 100%|██████████| 141/141 [00:00<00:00, 386B/s]

Upload successful: ./image_classifier/metadata.json (141B)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/asieduas/images/keras/image_classifier
